In [51]:
from fuzzysearch import find_near_matches
import pandas as pd
import regex as re
import spacy
from spacy.tokenizer import Tokenizer
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.cluster import DBSCAN
import en_core_web_sm
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re, string, unicodedata

In [52]:
def fileToDf(location1,location2): 
    #read data into a useful dataframe
    df1 = pd.read_table(location1, sep = ',')
    df2 = pd.read_table(location2, sep = ',')
    
    cmt_lst = df1.values[:,5].tolist()
    issid_lst = df1.values[:,1].tolist()
    
    lb_lst_uniq = df2.values[:,8].tolist()
    issid_lst2 = df2.values[:,1].tolist()
    
    # merge lists with different lengths 
    d1 = dict(zip(issid_lst2,lb_lst_uniq))
    lb_lst = []
    for iss in issid_lst:
        lb_lst.append(d1[iss])
        
        
    df = pd.DataFrame({'Content': cmt_lst,
                       'Issue_id': issid_lst,
                       'Label': lb_lst
                       })
    #print(df.head())
    df.shape
    return df

In [53]:
prefix_re = re.compile(r'''^[\[\("']''')
suffix_re = re.compile(r'''[\]\)"']$''')
infix_re = re.compile(r'''[-~]''')
simple_url_re = re.compile(r'''^https?://''')

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=simple_url_re.match)

def replaceUrls(dataframe):
    #replace urls (not effective for irregular urls)
    nlp = spacy.load('en_core_web_sm')
    nlp.tokenizer = custom_tokenizer(nlp)

    strList = []

    for cmt in dataframe['Content'].iteritems():
        msg = str(cmt)

        doc = nlp(msg)

        for i, token in enumerate(doc):
            if re.match(r'.*(http[s]:\/\/)stackoverflow.com.*', token.lemma_):
                token.lemma_ = 'SO-url'
            elif re.match(r'.*(http[s]:\/\/).*', token.lemma_):
                token.lemma_ = 'other-url'

        modifiedStr = ''
        for token in doc:
            modifiedStr += ' ' + token.lemma_
        strList.append(modifiedStr)
    dataframe['Content'] = strList
    #print(dataframe.head())
    dataframe.shape
    return dataframe

In [54]:
#helper functions to extract the sentences

def findSE(input):
    soStr = str(input)
    nlst = []
    if len(soStr) == 0:
        return nlst
    counter = 2
    wrt = 0
    str1 = ""
    str2 = ""
    while counter > 0:
        for i in range(len(soStr)):
            if soStr[i] == ",":
                wrt = 0
                counter -= 1
            if wrt == 1 and counter == 2:
                str1 += soStr[i]
            if wrt == 1 and counter == 1:
                str2 += soStr[i]
            if soStr[i] == "=":
                wrt = 1
    int1 = int(str1)
    int2 = int(str2)
    nlst.append(int1)
    nlst.append(int2)
    return nlst

def findBefore(cmtStr, wSize, start):
    counter = wSize
    beforeList = []
    beforeWord = ""
    pointer = start - 1
    while counter > 0:
        if pointer == -1:
            beforeList.insert(0, beforeWord)
            return beforeList
        if cmtStr[pointer] == " " and cmtStr[pointer-1] != " ":
            if beforeWord != "":
                beforeList.insert(0, beforeWord)
                counter -= 1
            if counter == 0:
                return beforeList
            beforeWord = ""
            pointer -= 1
        elif cmtStr[pointer] == " " and cmtStr[pointer-1] == " ":
            pointer -= 1
        else:
            beforeWord = cmtStr[pointer] + beforeWord
            pointer -= 1
    return beforeList

def findAfter(cmtStr, wSize, end):
    counter = wSize
    afterList = []
    afterWord = ""
    pointer = end + 1
    while counter > 0:
        if pointer == len(cmtStr):
            afterList.append(afterWord)
            return afterList
        if cmtStr[pointer] == " " and cmtStr[pointer+1] != " ":
            if afterWord != "":
                afterList.append(afterWord)
                counter -= 1
            if counter == 0:
                return afterList
            afterWord = ""
            pointer += 1
        elif cmtStr[pointer] == " " and cmtStr[pointer+1] == " ":
            pointer += 1
        else:
            afterWord += cmtStr[pointer]
            pointer += 1
    return afterList

def findWindow(cmtStr, wSize, start, end, keyWord):
    before = findBefore(cmtStr, wSize, start)
    after = findAfter(cmtStr, wSize, end)
    window = before + keyWord + after
    newStr = ''
    for i in window:
        newStr += ' ' + i
    return newStr


In [55]:
def extractWindow(dataframe):
    nlst = []
    isslst = []

    for cmt,isid in zip(dataframe['Content'],dataframe['Issue_id']):
        matches1 = find_near_matches('stack overflow', str(cmt).lower(), max_l_dist=2)
        matches2 = find_near_matches(' SO ', str(cmt), max_l_dist=0)
        matches3 = find_near_matches('so question', str(cmt).lower(), max_l_dist=0)
        matches4 = find_near_matches('so issue', str(cmt).lower(), max_l_dist=0)
        matches5 = find_near_matches('so forum', str(cmt).lower(), max_l_dist=0)
        #print(matches1)
        #print(matches2)
        #print(matches3)
        #print(matches4)
        #print(matches5)
        strCmt = str(cmt)
        if len(matches1) == 0 and len(matches2) == 0 and len(matches3) == 0 and len(matches4) == 0 and len(matches5) == 0:
            #nlst.append([])
            pass
        else:
            for i in range(len(matches1)):
                se = findSE(matches1[i])
                start = se[0]
                end = se[1]
                #print(start)
                #print(end)
                #print(cmt)
                currentWin = findWindow(strCmt, 5, start, end, ['stack overflow'])
                #print(currentWin)
                nlst.append(currentWin)
                if se != []:
                    isslst.append(isid)
            for i in range(len(matches2)):
                se = findSE(matches2[i])
                start = se[0]
                end = se[1]
                #print(start)
                #print(end)
                #print(cmt)
                currentWin = findWindow(strCmt, 5, start, end, ['SO'])
                #print(currentWin)
                nlst.append(currentWin)
                if se != []:
                    isslst.append(isid)
            for i in range(len(matches3)):
                se = findSE(matches3[i])
                start = se[0]
                end = se[1]
                #print(start)
                #print(end)
                #print(cmt)
                currentWin = findWindow(strCmt, 5, start, end, ['so question'])
                #print(currentWin)
                nlst.append(currentWin)
                if se != []:
                    isslst.append(isid)
            for i in range(len(matches4)):
                se = findSE(matches4[i])
                start = se[0]
                end = se[1]
                #print(start)
                #print(end)
                #print(cmt)
                currentWin = findWindow(strCmt, 5, start, end, ['so issue'])
                #print(currentWin)
                nlst.append(currentWin)
                if se != []:
                    isslst.append(isid)
            for i in range(len(matches5)):
                se = findSE(matches5[i])
                start = se[0]
                end = se[1]
                #print(start)
                #print(end)
                #print(cmt)
                currentWin = findWindow(strCmt, 5, start, end, ['so forum'])
                #print(currentWin)
                nlst.append(currentWin)
                if se != []:
                    isslst.append(isid)
            #nlst.append(currentRow)
            #currentRow = []
    #print(nlst)
    print(len(nlst))

    return nlst,isslst


In [56]:
def combined(location1,location2):
    dataframe = fileToDf(location1,location2)
    dataframe = replaceUrls(dataframe)
    sentenceLst,issid_lst = extractWindow(dataframe)
    return sentenceLst,issid_lst

In [57]:
#enter all data locations
locations = ['E:/qrf/Lab Stuff/newDataset/spring-boot_PRcomments.txt']
locations_of_q = ['E:/qrf/Lab Stuff/newDataset/spring-boot_PRs.txt']
allSentences = []
allissueid = []
for x,y in zip(locations,locations_of_q):
    newsentencelst,newisslst = combined(x,y)
    allSentences += newsentencelst
    allissueid += newisslst


d:\Anaconda3\envs\CISC880_SO_Directed_Issues\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
d:\Anaconda3\envs\CISC880_SO_Directed_Issues\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


  Content  Issue_id              Label
0             31532  status: duplicate
1             31532  status: duplicate
2             31541    status: invalid
3             31541    status: invalid
4             31541    status: invalid


In [ ]:
# embedding
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(allSentences, show_progress_bar=True)

#Compute cosine-similarity between all pairs of sentences
cosine_scores = util.cos_sim(embeddings, embeddings).numpy()

In [ ]:
# the similarity of a sentence with itself
print(cosine_scores[0][0])

# the similarity of a sentence with others
print(cosine_scores[0][1:6])

In [ ]:
# convers cosine similarity to distance, to be used later in the clustering algorithm
distance = np.abs(1 - np.abs(cosine_scores))

In [ ]:
# the distance of a sentence with itself
print(distance[0][0])

# the distance of a sentence with others
print(distance[0][1:6])

In [ ]:
db = DBSCAN(eps=0.5, min_samples=5, metric='precomputed').fit(distance)
labels = db.labels_

In [ ]:
from collections import Counter
for label, count in Counter(labels).items():
    print(label,count)

In [ ]:
labeled_sentences = pd.DataFrame({'issueid':allissueid, 'sentence':allSentences, 'label':labels})

In [ ]:
labeled_sentences.to_csv('spring-boot_labeled_sentences.csv')